# 1. Introducción a DataFrames:
## Conceptos básicos:
Los DataFrames son una abstracción de datos estructurados, organizados en filas y columnas, con un esquema definido. Esta estructura facilita la manipulación y el análisis de datos utilizando las APIs de Spark SQL.

## Creación de DataFrames:
- **Desde RDDs**: Los DataFrames pueden crearse a partir de RDDs (colecciones distribuidas de datos). La creación de un DataFrame desde un RDD permite trabajar con datos no estructurados transformándolos en un formato tabular.
- **Desde archivos**: Spark SQL permite la creación de DataFrames desde varios formatos de archivos, como CSV, JSON y Parquet. Puedes cargar estos archivos directamente en un DataFrame utilizando la API de Spark SQL. También se pueden usar otros formatos de archivo.
- **Desde tablas Hive**: Puedes crear DataFrames a partir de tablas existentes en Hive, aprovechando el metastore de Hive.
- **Otras fuentes**: Spark puede leer datos de diversas fuentes incluyendo bases de datos relacionales mediante JDBC, NoSQL, ORC, y otros sistemas de almacenamiento.


### 1. Cargar datos desde un RDD:
Para convertir un RDD en un DataFrame, se utiliza la función `toDF()` o `createDataFrame()`.

- **toDF()**: Infiere el esquema del DataFrame a partir del RDD, normalmente usado con una tupla o lista en Python.  https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.toDF.html (similar, la original no está documentada en https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)
- **createDataFrame()**: Permite especificar explícitamente el esquema (`StructType`) del DataFrame.  https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.createDataFrame.html

In [0]:
# Inicializamos sesión

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RDDtoDF").getOrCreate()


In [0]:
import sys
print("Python version: ", sys.version)

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print("Spark version: ", sc.version)


In [0]:
## Ejemplo con toDF():
rdd = spark.sparkContext.parallelize([("Alice", 34), ("Bob", 23)])
df = rdd.toDF(["name", "age"])
df.show()

## Ejemplo sobre rdd cargado de un fichero (error porque no consigue inferir el esquema)
lines = sc.textFile("dbfs:/FileStore/u.data")
df_peliculas = lines.toDF()
df_peliculas.show()

In [0]:
## Ejemplo con createDataFrame() especificando el esquema:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

esquema = StructType([
            StructField("usuario",IntegerType(),False),
            StructField("pelicula",IntegerType(),False),
            StructField("rating", IntegerType(),False),
            StructField("timestamp",IntegerType(),False)
])

lineas_rdd = sc.textFile("dbfs:/FileStore/u.data")

# Convertimos un rdd de filas en un rdd de listas de 4 strings
lineas_rdd_cadenas = lineas_rdd.map(lambda x: x.split())

# Como el esquema espera enteros, convertimos el rdd a listas de 4 enteros
lineas_rdd_enteros = lineas_rdd_cadenas.map(lambda x: [int(x[0]), int(x[1]), int(x[2]), int(x[3])])

df_peliculas = spark.createDataFrame(lineas_rdd_enteros,esquema)
df_peliculas.show()

In [0]:
# Mismo caso pero definiendo la función para el map
def leerFila(fila):
    lista = fila.split()
    return [int(lista[0]),int(lista[1]),int(lista[2]),int(lista[3])]

lineas_rdd_filas = sc.textFile("dbfs:/FileStore/u.data")
lineas_rdd_enteros2 = lineas_rdd_filas.map(leerFila)

df_peliculas2 = spark.createDataFrame(lineas_rdd_enteros2,esquema)
df_peliculas2.show()

### 2. Cargar datos desde ficheros CSV:

#### Sintaxis:
Se utiliza `spark.read.csv()`. Se pueden especificar opciones como `header` para indicar si el archivo tiene encabezado e `inferSchema` para que Spark infiera los tipos de datos.

- `header` indica si la primera línea del archivo CSV contiene los nombres de las columnas.
- `inferSchema` permite a Spark determinar automáticamente los tipos de datos de cada columna.

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.csv.html

#### Consideraciones
- **Rutas de archivos**: Asegúrate de proporcionar las rutas correctas a tus archivos CSV y Parquet.
- **Esquema**: Si no se utiliza `inferSchema` al leer archivos CSV, el esquema del DataFrame debe especificarse explícitamente.
- **DataFrames**: Los DataFrames proporcionan una forma de procesar y analizar datos estructurados. A diferencia de los RDDs, los DataFrames están basados en un esquema, es decir, conocen los nombres y tipos de las columnas de un conjunto de datos.

In [0]:
# Desde un fichero CSV
df=spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/olive.csv")
df.show()


### 3. Cargar datos desde ficheros Parquet:
Se utiliza `spark.read.parquet()`.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.parquet.html  

#### Consideraciones
- **Esquema**: El esquema se almacena en el mismo archivo.

In [0]:
# Desde un único fichero parquet
df=spark.read.parquet("dbfs:/FileStore/palm.parquet")
df.show(1000)


# 2. API de DataFrames:
Para realizar transformaciones en un DataFrame en Spark con Python, se utilizan diversas funciones que permiten modificar, seleccionar, o agregar datos. Esta es la sintaxis y ejemplos de uso de algunas de las transformaciones más comunes:


## Transformaciones:
- **select**: Permite seleccionar columnas específicas de un DataFrame.
- **filter**: Permite filtrar filas basadas en una condición.
- **withColumn**: Permite añadir nuevas columnas o modificar las existentes.
- **Otras transformaciones**: La API incluye otras transformaciones para manipular los datos como groupBy, sort y join. También permite crear funciones definidas por el usuario para manipulación personalizada de datos.

## Acciones:
- **show**: Muestra las primeras filas de un DataFrame.
- **count**: Cuenta el número de filas en un DataFrame.
- **collect**: Retorna todos los elementos de un DataFrame al driver (cuidado con el uso en grandes datasets).
- **Otras acciones**: Incluyen take, takeSample y describe para obtener información y estadísticas sobre los DataFrames.

## Consideraciones:

- **Inmutabilidad**: Los DataFrames son inmutables; cada transformación crea un nuevo DataFrame.
- **show()**: La función `show()` se utiliza para mostrar una muestra de los datos resultantes tras una transformación.
- **Importaciones**: Algunas funciones requieren importaciones adicionales desde `pyspark.sql.functions`, como `col`, `lit`, `expr`, `avg`, `count`, etc.
- **Expresiones SQL**: Puedes usar expresiones SQL con `expr()` y `selectExpr()` para transformaciones más complejas.
- **Columnas**: Las columnas se pueden referenciar usando su nombre como string, usando la notación de corchetes sobre el DataFrame o con la función `col()`.

### 1. select():
Se utiliza para seleccionar un subconjunto de columnas de un DataFrame. También se puede usar `selectExpr()` para seleccionar columnas con expresiones SQL.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.select.html

In [0]:
df=spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/olive.csv")

df.select("Country", "Year", "Production").show() 
df.select('*').show()
df.select(df.Country, (df.Production * 1000).alias('Production (Tm)')).show()


### 2. filter() o where():
Se utiliza para filtrar filas basadas en una condición. `filter()` y `where()` son sinónimos.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html

In [0]:
df.filter((df["Country"] == "Spain") & (df["Year"] < 1984)).select("Country", "Year", "Production").show()


### 3. withColumn():
Se utiliza para añadir una nueva columna o reemplazar una existente. La función `lit()` crea una columna con un valor literal y `expr()` permite usar expresiones SQL.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html

In [0]:
from pyspark.sql.functions import lit, expr
from datetime import datetime

df=spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/olive.csv")
df_Spain = df.filter((df["Country"] == "Spain") | (df["Country"] == "European Union")).select("Country", "Year", "Production")

df_Spain = df_Spain.withColumn("Production (Tm)", df_Spain["Production"]*1000)
df_Spain = df_Spain.withColumn("Region", expr("CASE WHEN Year <= 1990 THEN 'Spain' ELSE 'EU' END"))

current_year = datetime.now().year
df_Spain = df_Spain.withColumn("Diff_Years", lit(current_year) - df_Spain["Year"])

df_Spain.show(df_Spain.count())

### 4. withColumnRenamed():
Se utiliza para renombrar una columna existente.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html

In [0]:
df_Spain.withColumnRenamed("Country", "País").withColumnRenamed("Year", "Año").show()

### 5. groupBy():
Se utiliza para agrupar filas con valores iguales en una columna y realizar operaciones de agregación. Se combina con funciones de agregación como `count()`, `sum()`, `avg()`, `min()`, `max()`.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.groupBy.html

In [0]:
# Podemos realizarlo de dos formas: utilizando funciones de F o con diccionarios. 
# La primera es más clara y permite realizar varias agregaciones sobre la misma columna.

df=spark.read.parquet("dbfs:/FileStore/palm.parquet")

from pyspark.sql import functions as F
df.select(df.Country, df.Year, df.Production) \
    .groupBy("Country") \
    .agg(
        F.sum("Production").alias("TotalProd"),
        F.max("Production").alias("MaxProd")
    ) \
    .orderBy("TotalProd",ascending=False) \
.show(4)

df.select(df.Country, df.Year, df.Production) \
    .groupBy("Country") \
    .agg({"Production": "sum"}) \
    .withColumnRenamed("sum(Production)", "TotalProd") \
    .orderBy("TotalProd", ascending=False) \
.show(4)

### 6. sort() o orderBy():
Se utiliza para ordenar las filas del DataFrame. `sort()` y `orderBy()` son equivalentes y pueden usar el orden ascendente (`asc`) o descendente (`desc`).  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sort.html  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.orderBy.html

In [0]:
from pyspark.sql import functions as F
df=spark.read.parquet("dbfs:/FileStore/palm.parquet")
df = df.select(df.Country, df.Year, df.Production).groupBy("Country","Year").agg(F.sum("Production").alias("TotalProd"))
df.sort("Country").show(5)
df.sort(df["Country"].desc()).show(5)
df.orderBy("TotalProd").show(5)
df.orderBy(df["TotalProd"].desc()).show(5)

### 7. drop():
Se utiliza para eliminar una o varias columnas del DataFrame.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.drop.html

In [0]:
df_Spain.show(5)
df_Spain_reducido = df_Spain.drop("Year", "Diff_Years")
df_Spain_reducido.show(5)

### 8. distinct():
Se utiliza para eliminar las filas duplicadas del DataFrame.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.distinct.html

In [0]:
df_Spain_reducido.distinct().show()


### 9. join():
Se utiliza para combinar dos DataFrames basados en una o varias columnas en común. Se puede especificar el tipo de join: 'inner', 'outer', 'left_outer', 'right_outer', o 'leftsemi'.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.join.html

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import functions as F

# Definir el esquema del DataFrame de ratings
esquemaRatings = StructType([
    StructField("UserID", IntegerType(), True),
    StructField("MovieID", IntegerType(), True),
    StructField("Rating", IntegerType(), True),
    StructField("Timestamp", IntegerType(), True)
])

# Cargar el archivo de ratings u-data en un DataFrame, con separador el tabulador \t
dfRatings = spark.read.csv("dbfs:/FileStore/u.data", sep="\t", schema=esquemaRatings, header=False)

# Definir esquema para el DataFrame de películas
esquemaPeliculas = StructType([
    StructField("MovieID", IntegerType(), True),
    StructField("Title", StringType(), True),
    StructField("ReleaseDate", StringType(), True),
    StructField("EmptyColumn", StringType(), True),
    StructField("IMDB_URL", StringType(), True),
    StructField("Unknown", IntegerType(), True),
    StructField("Action", IntegerType(), True),
    StructField("Adventure", IntegerType(), True),
    StructField("Animation", IntegerType(), True),
    StructField("Children", IntegerType(), True),
    StructField("Comedy", IntegerType(), True),
    StructField("Crime", IntegerType(), True),
    StructField("Documentary", IntegerType(), True),
    StructField("Drama", IntegerType(), True),
    StructField("Fantasy", IntegerType(), True),
    StructField("FilmNoir", IntegerType(), True),
    StructField("Horror", IntegerType(), True),
    StructField("Musical", IntegerType(), True),
    StructField("Mystery", IntegerType(), True),
    StructField("Romance", IntegerType(), True),
    StructField("SciFi", IntegerType(), True),
    StructField("Thriller", IntegerType(), True),
    StructField("War", IntegerType(), True),
    StructField("Western", IntegerType(), True)
])

# Cargar el archivo de películas en un DataFrame, con separador |
dfPeliculas = spark.read.csv("dbfs:/FileStore/u.item", sep="|", schema=esquemaPeliculas, header=False)

# Mostrar las 10 películas con más votos
dfRatingsNombres = dfRatings.join(dfPeliculas,on="MovieID",how="inner")
dfRatingsAgrupados = dfRatingsNombres.groupBy("Title").agg(F.count("Title").alias("Ratings")).orderBy("Ratings",ascending=False)
dfRatingsAgrupados.show(10)


### 10. union() o unionAll():
Se utiliza para combinar dos DataFrames con las mismas columnas. `union()` elimina duplicados, `unionAll()` no.  
    https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html
y https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.unionAll.html

In [0]:
# Crear dos dataframes con los mismos campos (películas más veces puntuadas y películas mejor puntuadas)
# Hacer la unión y mostrarlos.

# Películas más veces puntuadas
# dfRatingsBest (Title, Ratings, MeanRating)

# Películas mejor puntuadas, con más de 100 votos
# dfRatingsTop (Title, Ratings, MeanRating)

# dfRatingsBest.union(dfRatingsTop).show()



### 11. map():
Se utiliza para aplicar una función a cada fila del DataFrame, convirtiéndolo a RDD.  
La función map() se aplica a RDDs (Resilient Distributed Datasets), no directamente a DataFrames. Para usar map en un DataFrame, primero debes convertirlo a un RDD usando df.rdd.   
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html


In [0]:
# Obtener una lista de nombres de películas y un diccionario con el número de votos en cada puntuación:

# Nos quedamos con las columnas MovieID y Rating
dfRatings = dfRatings.select("MovieID", "Rating")

# Convertir el DataFrame de ratings a un RDD de filas
rddFilas = dfRatings.rdd

# Convertir el RDD de filas a un RDD de tuplas
rddTuplas = rddFilas.map(lambda fila: (fila[0], (fila[1],1)))

# Función para crear un diccionario con el número de votos para cada puntuación
def crearRatingDict(tuplas):
    RatingDict = {}
    for rating, cont in tuplas:
        if rating in RatingDict:
            RatingDict[rating] += cont
        else:
            RatingDict[rating] = cont
    return RatingDict

# Agrupar por MovieID y agregar las puntuaciones
rddRatingsAgrupados = rddTuplas.groupByKey().mapValues(crearRatingDict)

# Volver a convertir a dataframe y hacer join con películas para obtener el nombre

# Mostrar 10 películas con su nombre y puntuaciones
